# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Import Molecule Class
from src.base_molecule import Molecule

In [3]:
# Creates a molecule. You can use a list, a dictionary (the key MUST be "atoms")
# or another molecule object (see below)

# 1 Option
water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_molecule = Molecule(water)

# 2 Option
water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}
water_molecule = Molecule(water_dict)

h2_dict= {"atoms": [("H", 0, 0, 0), ("H", 0.50000, 0, 0)]}


In [4]:
## See initial molecule.

import py3Dmol

water_molecule = Molecule(water_dict)
water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
bi_water = water_molecule.add_fragments(water_dict)


xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(bi_water),'xyz')
xyz_view.setStyle({'stick':{}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
from copy import deepcopy
from numpy import random
import numpy as np

def random_move(initial_cluster : Molecule, fragment = 0):
    # Random step of every fragment, staying inside sphere
    random_gen = random.default_rng()

    #Approximately 3 bohr radius for each atom
    sphere_radius = initial_cluster.total_atoms*1.5*0.5


    #All fragments will be moved AND rotated
    final_cluster_serie_xyz = ""


    final_cluster = deepcopy(initial_cluster)
    for j in range(0,final_cluster.total_fragments):
        # angle between [0, 360)
        ax = random_gen.uniform() * 360
        ay = random_gen.uniform() * 360
        az = random_gen.uniform() * 360

        # moving between [-move, +move]
        move = sphere_radius / 1.6
        tx = move * (random_gen.uniform() - 0.5 )
        ty = move * (random_gen.uniform() - 0.5 )
        tz = move * (random_gen.uniform() - 0.5 )

        final_cluster = final_cluster.translate(0, x=tx, y=ty, z=tz)
        final_cluster = final_cluster.rotate(final_cluster.total_fragments-1, x=ax, y=ay, z=az)

    # saving coordinates as a string 
    final_cluster_serie_xyz += final_cluster.xyz

    #Checking if any fragments falls out of the sphere (ALREADY CHECKED)
    import math as m
    if m.sqrt(final_cluster.coordinates[1][1]*final_cluster.coordinates[1][1]+final_cluster.coordinates[1][2]*final_cluster.coordinates[1][2]+final_cluster.coordinates[1][3]*final_cluster.coordinates[1][3]) > sphere_radius:
        print("un fragmento está cayendo fuera de la esfera")
    return final_cluster


def controled_move(initial_cluster : Molecule, displacements, angular_rotations):

    final_cluster = deepcopy(initial_cluster)

    for i in range(final_cluster.total_fragments):

        if final_cluster.total_fragments > 2:
            final_cluster = final_cluster.translate(0, x=displacements[i][0], y=displacements[i][1], z=displacements[i][2])
            final_cluster = final_cluster.rotate(final_cluster.total_fragments-1, x=angular_rotations[i][0], y=angular_rotations[i][0], z=angular_rotations[i][0])
        elif final_cluster.total_fragments == 2:
            final_cluster = final_cluster.translate(final_cluster.total_fragments-1, x=displacements[i][0], y=displacements[i][1], z=displacements[i][2])
            final_cluster = final_cluster.rotate(final_cluster.total_fragments-1, x=angular_rotations[i][0], y=angular_rotations[i][0], z=angular_rotations[i][0])
        else:
            pass

    return final_cluster


#Visualization
def visualization_random_process(serie_string : str, n_atoms):
    #Approximately 3 bohr radius for each atom
    sphere_radius = n_atoms*1.5*0.5
    xyz_view = py3Dmol.view(width=1000,height=700)
    xyz_view.addModelsAsFrames(serie_string,'xyz')
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
        'radius': sphere_radius , 
        'color' :'yellow',
        'alpha': 0.5,
        })

    xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 10})
    xyz_view.zoomTo()
    xyz_view.show()


#### Funciones para combinar con shgo:
from scipy.optimize import shgo
from pyscf import gto, scf


# shgo
def optimization_shgo(cluster_of_interest):

    sphere_radius = cluster_of_interest.total_atoms*1.5*0.5
    discretization = sphere_radius / 1.6
    bounds = np.array([(-discretization,discretization),(-discretization,discretization),(-discretization,discretization),(0,360),(0,360),(0,360)])
    shgo_tolerance_dict = {"ftol" : 1e-6} # Default is 1e-12
    shgo_options_dict = {"options" : shgo_tolerance_dict}
    bounds_full = np.repeat(bounds,cluster_of_interest.total_fragments-1,axis=0)


    n=32 #128 default for sobol. Tiene que ser potencia de 2. Simplicial doesnt use it...
    #sobol, simplicial are the sampling methods

    minimos = shgo(optimization_kernel, bounds=bounds_full, minimizer_kwargs=shgo_options_dict, sampling_method='sobol',n=16, iters=1)


    return minimos



# Dual Annealing
def optimization_dual_annealing(cluster_of_interest):
    from scipy.optimize import dual_annealing

    sphere_radius = cluster_of_interest.total_atoms*1.5*0.5
    discretization = sphere_radius / 1.6
    bounds = np.array([(-discretization,discretization),(-discretization,discretization),(-discretization,discretization),(0,360),(0,360),(0,360)])

    bounds_full = np.repeat(bounds,cluster_of_interest.total_fragments-1,axis=0)
    bounds_full=np.array(bounds_full)

    lista_moves=[]
    lista_values=[]
    for iteracion in range (0,8):
        minimos = dual_annealing(optimization_kernel,bounds=bounds_full, maxiter=6, initial_temp=5230, local_search_options={"method" : 'BFGS'},no_local_search=True , restart_temp_ratio = 0.9)

        lista_moves.append(minimos.x)
        lista_values.append(minimos.fun)


    return lista_values, lista_moves

# Basinhopping
def optimization_juan_outer_3(cluster_of_interest):
    #from scipy.optimize import basinhopping
    # Este intenté por todos los medios, pero no funcionó bien... no movía prácticamente las variables, desde el vector inicial
        pass



def optimization_kernel(x):
#def optimization_juan_inner(x,*kwargs): #to test if I can send some objects here
    global serie_of_interest_xyz_last
    #No tengo modos de meter como argumento la molécula... 
    displacements= [ np.zeros(3) ]
    angular_rotations= [ np.zeros(3) ]
    for i in range(cluster_of_interest.total_fragments-1):
        displacements.append(x[i*3:i*3+3])
        angular_rotations.append(x[(cluster_of_interest.total_fragments-1+i)*3:(cluster_of_interest.total_fragments+i)*3])
    candidate_to_test = controled_move(cluster_of_interest, displacements, angular_rotations)
    serie_of_interest_xyz_last += candidate_to_test.xyz
    mol = gto.Mole()
    mol.fromstring(str(candidate_to_test))
    mol.build(basis = 'sto-3g')
    #RHF
    mf = scf.RHF(mol)
    mf.kernel()

    #DFT
    #from pyscf import dft
    #mf_hf = dft.RKS(mol)
    #mf_hf.xc = 'pbe,pbe'
    #mf_hf.kernel()
    return mf.energy_tot()





In [ ]:
#### Testing random moves inside sphere
total_steps = 10

cluster_of_interest = Molecule(
    water,
    water,
    water,
    water
)

#All fragments will be moved AND rotated
serie_of_interest_xyz = ""

for i in range(total_steps):
    serie_of_interest_xyz += random_move(cluster_of_interest).xyz

visualization_random_process(serie_of_interest_xyz,cluster_of_interest.total_atoms)

In [ ]:
# AMCESS test

cluster_of_interest = Molecule(
    water,
    water,
    water,
    water
)

shgo_bool = True
dual_annealing_bool = False


#Armo la configuración inicial para los fragmentos,
#a partir de la cual se efectúan rotaciones y traslaciones
#No pueden estar juntos inicialmente: el SCF saldría con error

delta=0.5
variaciones=[0,-delta,delta]

i,j,k = 0, 0, 0
if (shgo_bool):
    for A in range(cluster_of_interest.total_fragments):
        cluster_of_interest = cluster_of_interest.translate(0, x=variaciones[i], y=variaciones[j], z=variaciones[k])
        if ((A+1) % 3 == 0):
                k=0
                if ((A+1) % 9 == 0):
                        j=0
                        if ((A+1) % 27 == 0):
                            i=0
                            print("STOP. For",cluster_of_interest.total_fragments,"or more fragments this should be revised")
                            break
                        else:
                            i=i+1
                else:
                        j=j+1
        else:
                k=k+1


serie_of_interest_xyz_last = "" # Variable global...

if (shgo_bool):
    minimos = optimization_shgo(cluster_of_interest)
    print(minimos.xl, minimos.funl)

if (dual_annealing_bool):
    minimos , movimientos = optimization_dual_annealing(cluster_of_interest)



In [30]:
#Ploteo todos los puntos donde se evaluó la energía
visualization_random_process(serie_of_interest_xyz_last,cluster_of_interest.total_atoms)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [31]:


candidates=[]
candidates_coord=[]

if (shgo):

        for step in range(len(minimos.xl)):
                desplazamientos = [ np.zeros(3) ]
                rotaciones = [ np.zeros(3) ]
                for i in range(cluster_of_interest.total_fragments-1):
                        desplazamientos.append(minimos.xl[step][i*3:i*3+3])
                        rotaciones.append(minimos.xl[step][(cluster_of_interest.total_fragments-1+i)*3:(cluster_of_interest.total_fragments+i)*3])


        for i in range(len(minimos.xl)):
                retrieved_candidated = controled_move(cluster_of_interest, desplazamientos, rotaciones)
                candidates.append(retrieved_candidated)
                candidates_coord.append(retrieved_candidated.cartesian_coordinates)
        if len(minimos.xl) > 1:
                #Elimino candidatos que sean iguales (coordenadas todas muy parecidas)
                for i in range(len(minimos.xl)-1):
                        for k in range(i+1,len(candidates)):
                                if np.all(np.isclose(candidates[i].cartesian_coordinates, candidates[k].cartesian_coordinates,atol=0.05)):
                                        candidates = np.delete(candidates,[k]) #valores muy cercanos
                                else:
                                        pass


if (dual_annealing):
        for step in range(len(movimientos)):
                desplazamientos = [ np.zeros(3) ]
                rotaciones = [ np.zeros(3) ]
                for i in range(cluster_of_interest.total_fragments-1):
                        desplazamientos.append(movimientos[step][i*3:i*3+3])
                        rotaciones.append(movimientos[step][(cluster_of_interest.total_fragments-1+i)*3:(cluster_of_interest.total_fragments+i)*3])


                retrieved_candidated = controled_move(cluster_of_interest, desplazamientos, rotaciones)
                candidates.append(retrieved_candidated)
                candidates_coord.append(retrieved_candidated.cartesian_coordinates)


In [32]:
#Save results on file

#Clean file
with open('water.xyz', 'w') as f:
                f.write(" ")


for i in range(len(candidates)):
        mol = gto.Mole()
        mol.fromstring(str(candidates[i]))
        mol.build()

        mf = scf.RHF(mol)
        mf.kernel()
        energia = mf.energy_tot()


        with open('water.xyz', 'a') as f:

                f.write("---------------    Candidate %.0f    ---------------\n" % int(i+1))
                f.write(str(candidates[i]))
                f.write("Energy: %.6f \n" % mf.energy_tot())

converged SCF energy = -299.725611718808


In [33]:
## See Final molecule.

import py3Dmol

for i in range(len(candidates)):
    xyz_view = py3Dmol.view(width=300,height=200)
    xyz_view.addModel(str(candidates[i]),'xyz')
    #xyz_view.setStyle({'stick':{}})
    xyz_view.setStyle({'stick': {'radius': .15}, 'sphere': {'scale': 0.3}})

    xyz_view.zoomTo()
    xyz_view.show()



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [15]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj
